# Import libraries and modules
We will be using the following modules and libraries in our program:

- **math** - to access special mathematical functions
- **csv** - to create the submission file
- **collections** - to use specialized container data types
- **pandas** - to perform some data manipulation
- **NumPy**  - to work efficiently with large arrays

In [ ]:
# Python standard library modules
import math
import csv
import collections

# Open-source, BSD-licensed libraries
import pandas as pd
import numpy as np

# Subclass of the built-in Python dictionary that provides a default value for a non-existent key
from collections import defaultdict

# Set number of rows to be displayed
This pertains to the number of rows displayed when the data are presented in tabular form using pandas.

In [ ]:
# Number of rows to be displayed
NUM_ROWS = 2000

pd.set_option('display.max_rows', NUM_ROWS)
pd.set_option('display.min_rows', NUM_ROWS)

<br>

# Prepare data for processing
Before going to the core of the problem, we have to perform some preliminaries to make the actual data processing phase more systematic and efficient.

## 1. Load the dataset
The first step is, of course, to load the dataset. 

In [ ]:
data_raw = pd.read_csv('order_brush_order.csv')

It is a good idea to manipulate a **copy** of the dataset in order to avoid modifying the contents of the original.

In [ ]:
data = data_raw.copy(deep = True)

We display the data in tabular form to check if the dataset has been loaded successfully. This also gives us an initial idea of its contents, the format or manner in which they are stored, etc.

In [ ]:
data

## 2. Remove duplicate rows
Since each transaction is assigned a unique orderid, we discard duplicates in our dataset since they are most likely instances of misrecording or misencoding.

In [ ]:
data = data.drop_duplicates()

We display the data (with duplicates removed) in tabular form. 

In [ ]:
data

Note that the number of rows after performing duplication removal is the same as in the previous step. 
In other words, there are no duplicate entries.

## 3. Sort data
The idea behind sorting the data comes from the problem statement:
- We are interested in the instances of order brushing **per shop**.
- For each shop, we need to identify **time windows** when order brushing was conducted.

Therefore, we perform two-stage sorting: based on the shop IDs first, then based on the timestamps of the transactions (from earliest to latest).

In [ ]:
data.sort_values(by = ['shopid', 'event_time'], inplace = True, ignore_index = True)

We display the sorted data in tabular form.

In [ ]:
data

## 4. Convert event_time to datetime object
Since we are interested in identifying one-hour intervals, we need to convert the entries in the event_time column into actual datetime objects.

In [ ]:
# Explicit specification of format speeds up execution time.
data['event_time'] = pd.to_datetime(data['event_time'], format = '%Y-%m-%d %H:%M:%S')

We check if the conversion is successful by printing the data type of the first event_time entry.

In [ ]:
print(type(data.at[0, 'event_time']))

## 5. Create NumPy arrays for the columns
Our algorithm hinges on iterating over the rows of the dataset. However, doing this with pandas is an anti-pattern, significantly slowing down the execution time of our program. Therefore, it is more efficient if we process the columns as parallel NumPy arrays instead.

In [ ]:
# NumPy array for orderid
orderid_list = data['orderid'].to_numpy()

# NumPy array for shopid
shopid_list = data['shopid'].to_numpy()

# NumPy array for userid
userid_list = data['userid'].to_numpy()

# NumPy array for event_time
event_time_list = data['event_time'].to_numpy()

We check if the NumPy arrays have been successfully created.

In [ ]:
print(type(orderid_list))
print(type(shopid_list))
print(type(userid_list))
print(type(event_time_list))

<br>

# Process data

With all the preliminary preparations finished, we now proceed to the core of the order brushing detection task.

## 1. Set the "constants"
Setting "constants" is a good programming practice that discards so-called "magic numbers," which greatly detract code readability. In addition, it makes our program more maintainable and flexible to changes.

Based on the problem statement:
- Order brushing must be checked for every one-hour window (1 hour = 3600 seconds).
- Order brushing is conducted if the concentration rate is at least 3.0

Note that, if there are $n$ entries, then the maximum concentration rate is $n$, which occurs when all the orders come from a single buyer. Therefore, the minimum number of entries needed to reach a concentration rate of at least $x$ is $\lceil x \rceil$, the least integer that is greater than or equal to $x$.

In [ ]:
# Number of seconds in a possible order brushing period
BRUSH_NUM_SECONDS = 3600

# Minimum concentration rate for order brushing
BRUSH_CONCEN_RATE = 3.0

# Minimum number of entries for order brushing
BRUSH_NUM_ENTRIES = math.ceil(BRUSH_CONCEN_RATE)

## 2. Create some functions
Following the "divide-and-conquer" approach in programming, functions help make our program more modular. Note that these functions take advantage of the fact that the data have already been sorted.

### a. Function that creates the array containing the indices of the first entries per shop

**Parameter:**
- *shopid_list* is the array containing the shop IDs in the dataset

**Return:**
- array containing the indices of the first entries per shop

Since this function is also used to determine the indices of the last entries per shop, a "dummy shop" was added at the end in order to get the index of the last entry of the last shop. Therefore, the last element of the array returned by this function is (the index of the last entry of the last shop + 1).

In [ ]:
def make_shopgrp_idx_list(shopid_list):
    shopgrp_idx_list = np.flatnonzero(np.concatenate(([True], shopid_list[1:] != shopid_list[:-1])))
    shopgrp_idx_list = np.concatenate((shopgrp_idx_list, [len(shopid_list)]))
    
    return shopgrp_idx_list

Despite its brevity, the algorithm presented in this function &mdash; which takes advantage of the efficiency of NumPy functions &mdash; is not immediately intuitive. Hence, we expound on the logic behind its implementation:

*First line in the function body*
- Perform an element-by-element comparison of shopid_list excluding the first element and shopid_list excluding the last element. 
- Since we are interested in the indices of the first entries per shop, we check for **inequality**. If they are not equal, then the result of this comparison is *True*; otherwise, the result is *False*. 
- Store the results in an array.
- Prepend *True* to this array to account for the first entry of the first shop.
- Note that the indices of the *True* elements correspond to the indices of the first entries per shop. 
- Store these indices in an array using the NumPy flatnonzero() function (*True* is a nonzero value in programming).

*Second line in the function body*
- Add the "dummy shop." Since the index of the last entry of the last shop is (the number of elements in shopid_list &ndash; 1), the index of the first entry of this "dummy shop" is equal to the number of elements in shopid_list.

### b. Function that checks if the start time and end time are within a given number of seconds

**Parameters:**
- *start_time* is the start time
- *end_time* is the end time
- *num_seconds* is the maximum difference between the start time and end time (in terms of seconds)

**Return:**
- *True*, if the start and end time are within the given number of seconds
- *False*, otherwise

In [ ]:
def is_within_time(start_time, end_time, num_seconds):
    
    # Since event_time_list is a NumPy array, the number of seconds has to be converted to a NumPy timedelta object.
    return end_time - start_time <= np.timedelta64(num_seconds, 's')

### c. Function that computes the concentration rate in a given period 

**Parameters:**
- *userid_list* is the array containing the user IDs of the buyers in the dataset
- *start_time_idx* is the index of the start time
- *end_time_idx* is the index of the end time

**Return:**
- concentration rate in the given period:
$$\text{concentration rate} = \dfrac{\text{number of orders}}{\text{number of unique users}}$$

In [ ]:
def concen_rate(userid_list, start_time_idx, end_time_idx):
    
    # Count the number of unique users.
    num_unique_users = len(np.unique(userid_list[start_time_idx : end_time_idx + 1]))
    
    # Count the number of orders.
    num_orders = end_time_idx - start_time_idx + 1 
    
    # Compute and return the concentration rate.
    return num_orders / num_unique_users

### d. Function that checks if a given period is a possible order brushing period

**Precondition:**
- Assume that the start time and end time are within the time window for a possible order brushing period.
- Assume that the index of the start time is greater than the index of the first entry of the current shop. **(The case when the indices are equal is handled in the main program routine.)**

**Parameters:**
- *event_time_list* is the array containing the timestamps in the dataset
- *start_time_idx* is the index of the start time
- *end_time_idx* is the index of the end time
- *next_shopgrp_idx* is the index of the first entry of the next shop
- *brush_num_seconds* is the number of seconds in a possible order brushing period

**Return:**
- *True*, if the given period is a possible order brushing period
- *False*, otherwise

In [ ]:
def is_possible_brush_time(event_time_list, start_time_idx, end_time_idx, next_shopgrp_idx, brush_num_seconds):
    
    # It is important to evaluate the first expression before evaluating the second one.
    # Otherwise, we will run into the error of accessing an element outside the bounds of the current shop.
    return end_time_idx == next_shopgrp_idx - 1 \
        or not is_within_time(event_time_list[start_time_idx - 1], event_time_list[end_time_idx + 1], brush_num_seconds + 1)

Despite its brevity, this is arguably the most difficult subroutine to write. The challenge is to determine if a given period constitutes an order brushing period *without checking through all possible 1-hour windows*. Hence, we expound on the logic behind its implementation.

We begin by introducing some notation to formalize our logic:
- <b>start_time<sub>order</sub></b> is the start time referred to *by default*, i.e., the time of the first order in the one-hour period being investigated.
- <b>end_time<sub>order</sub></b> is the end time referred to *by default*, i.e., the time of the last order in the one-hour period being investigated.
- <b>start_time<sub>order &ndash; 1 </sub></b> is the time of the order immediately before the order referred to in start_time<sub>order</sub>, *provided it exists.*
- <b>end_time<sub>order + 1 </sub></b> is the time of the order immediately after the order referred to in end_time<sub>order</sub>, *provided it exists.*
- <b>start_time<sub>true</sub></b> is the start time of the one-hour period being investigated.
- <b>end_time<sub>true</sub></b> is the end time of the one-hour period being investigated.

The following conditions should hold true:
- <b>start_time<sub>order &ndash; 1</sub> < start_time<sub>true</sub> &#8804; start_time<sub>order</sub> < end_time<sub>order</sub> &#8804; end_time<sub>true</sub> < end_time<sub>order + 1</sub></b>
- <b>start_time<sub>order</sub> and end_time<sub>order</sub> are within one hour.</b> This is a precondition for calling this function, so we do not need to worry about this.
- <b>start_time<sub>true</sub> and end_time<sub>true</sub> are within one hour.</b>

The function returns *True* if at least one of these conditions holds true:
- <b>end_time<sub>order</sub> is the timestamp of the last entry of the current shop.</b> This takes care of the case when end_time<sub>order + 1</sub> does not exist. Thanks to the precondition, setting start_time<sub>true</sub> = start_time<sub>order</sub> and setting end_time<sub>true</sub> to 1 hour after start_time<sub>true</sub> will satisfy the conditions above.


- <b>start_time<sub>order &ndash; 1</sub> and end_time<sub>order + 1</sub> are not within one hour and one second.</b> end_time<sub>true</sub> < end_time<sub>order + 1 </sub> implies that end_time<sub>order + 1 </sub> is one second after end_time<sub>true</sub> at the earliest. Analogously, start_time<sub>order &ndash; 1</sub> < start_time<sub>true</sub> implies that start_time<sub>order &ndash; 1</sub> is one second before start_time<sub>true</sub> at the latest. Taking into account that the difference between start_time<sub>true</sub> and end_time<sub>true</sub> is at most 3600 seconds, it follows that the difference between start_time<sub>order &ndash; 1</sub> and end_time<sub>order + 1 </sub> is at least 3602 seconds.


Note that this logic still holds even if we change the length of the time window. Reiterating the second precondition, the case when start_time<sub> order &ndash; 1</sub> does not exist is deferred to the main program routine.

### e. Function that checks if a given concentration rate meets the minimum for order brushing

**Parameters:**
- *concen_rate* is the given concentration rate
- *brush_concen_rate* is the minimum concentration rate for order brushing

**Return:**
- *True*, if the given concentration rate meets the minimum for order brushing
- *False*, otherwise

In [ ]:
def is_order_brushing(concen_rate, brush_concen_rate):
    return concen_rate >= brush_concen_rate

### f. Function that identifies the suspicious buyers in a shop

**Parameter:**
- *transact_dict* is the dictionary containing the user IDs as keys and the list of associated order IDs as values

**Return:**
- list of the user IDs of the suspicious buyers in an order brushing shop
- [0], if the shop is not found to have conducted order brushing

The suspicious buyers in an order brushing shop refer to those who ordered the highest number of products during order brushing periods.

In [ ]:
def get_suspicious(transact_dict):
    
    # An empty transact_dict implies that the shop is not found to have conducted order brushing.
    if not transact_dict:
        suspicious_list = [0]
    
    else:
        # Get the highest number of products ordered by a single user.
        max_num_orders = 0
        for orders in transact_dict.values():
            if len(orders) > max_num_orders:
                max_num_orders = len(orders)

        # Get the user IDs of the suspicious buyers.
        suspicious_list = [userid for userid, orders in transact_dict.items() if len(orders) == max_num_orders]

        # The task specifies that the user IDs of the suspicious buyers should be listed in ascending order.
        if len(suspicious_list) > 1:
            suspicious_list.sort()
    
    return suspicious_list

### g. Function that assigns the suspicious buyers to the master shop dictionary

**Parameters:**
- *shop_dict* is the master shop dictionary containing the shop IDs as keys and the list of associated suspicious buyers as values
- *suspicious_list* is the list of the user IDs of the suspicious buyers in an order brushing shop ([0] if the shop is not found to have conducted order brushing)
- *shopid_list* is the list of shop IDs in the dataset
- *curr_shopgrp_idx* is the index of the first entry of the current shop

In [ ]:
def assign_suspicious(shop_dict, suspicious_list, shopid_list, curr_shopgrp_idx):
    shop_dict[shopid_list[curr_shopgrp_idx]] = suspicious_list

## 3. Solve the problem
With the subroutines above completing our arsenal, we are now ready to tackle the problem.

### a. Variable initialization
We initialize the containers that we will be using in the main routine.

In [ ]:
# Array containing the indices of the first entries per shop
shopgrp_idx_list = make_shopgrp_idx_list(shopid_list)

# Dictionary containing the shop IDs as keys and the list of associated suspicious buyers as values
shop_dict = {}

# Dictionary containing the user IDs as keys and the set of associated order IDs as values
transact_dict = defaultdict(set)

### b. Main routine
In writing the main routine to solve the problem, it is important to point out that having the data sorted is a precondition.

**Moreover, the case when start_time<sub>order &ndash; 1</sub> does not exist is handled in this step.** This warrants handling the first entry of each shop separately from the rest. As long as start_time<sub>order</sub> (the first entry of each shop) and end_time<sub>order</sub> are within 1 hour, then the conditions for a possible order brushing period are satisfied since we can set end_time<sub>true</sub> = end_time<sub>order</sub> and set start_time<sub>true</sub> to 1 hour before end_time<sub>true</sub>.

Note that this logic still holds even if we change the length of the time window.

In [ ]:
# Iterate over every shop. Each iteration corresponds to a unique shop ID.
for i, curr_shopgrp_idx in enumerate(shopgrp_idx_list[:-1]):  
    
    # Index of the first entry of the next shop
    next_shopgrp_idx = shopgrp_idx_list[i + 1]
    
    
    # HANDLE THE FIRST ENTRY SEPARATELY AS EXPLAINED ABOVE.
    start_time_idx = curr_shopgrp_idx
    start_time = event_time_list[start_time_idx]
    
    # Iterate over every end time.
    # We skip the first (BRUSH_NUM_ENTRIES - 1) entries since at least BRUSH_NUM_ENTRIES are needed
    #     to constitute an order brushing period.
    for end_time_idx, end_time in enumerate(event_time_list[start_time_idx + BRUSH_NUM_ENTRIES - 1 : next_shopgrp_idx], 
                                            start_time_idx + BRUSH_NUM_ENTRIES - 1):
        
        # If the start time and end time are already beyond the time window for an order brushing period,
        #     we exit the loop and proceed to the next start time.
        if not is_within_time(start_time, end_time, BRUSH_NUM_SECONDS):
            break
        
        # Check if order brushing was conducted. 
        if is_order_brushing(concen_rate(userid_list, start_time_idx, end_time_idx), BRUSH_CONCEN_RATE):
            
            # Update transact_dict of the shop by iterating over all the entries in the period.
            for userid, orderid in zip(userid_list[start_time_idx : end_time_idx + 1], 
                                       orderid_list[start_time_idx : end_time_idx + 1]):
                transact_dict[userid].add(orderid)
                    
    
    # HANDLE THE REMAINING ENTRIES.
    
    # Iterate over every start time.
    # We only iterate until the (BRUSH_NUM_ENTRIES)th-to-the-last timestamp 
    #     since at least BRUSH_NUM_ENTRIES are needed to constitute an order brushing period.
    for start_time_idx, start_time in enumerate(event_time_list[curr_shopgrp_idx + 1 : next_shopgrp_idx - BRUSH_NUM_ENTRIES + 1],
                                                curr_shopgrp_idx + 1):
        
        # Iterate over every end time.
        # We skip the first (BRUSH_NUM_ENTRIES - 1) entries since at least BRUSH_NUM_ENTRIES are needed
        #     to constitute an order brushing period.
        for end_time_idx, end_time in enumerate(event_time_list[start_time_idx + BRUSH_NUM_ENTRIES - 1 : next_shopgrp_idx], 
                                                start_time_idx + BRUSH_NUM_ENTRIES - 1):
            
            # If the start time and end time are already beyond the time window for an order brushing period,
            #     we exit the loop and proceed to the next start time.
            if not is_within_time(start_time, end_time, BRUSH_NUM_SECONDS):
                break
            
            # Check if the period is a possible order brushing period and if order brushing was conducted.
            if is_possible_brush_time(event_time_list, start_time_idx, end_time_idx, next_shopgrp_idx, BRUSH_NUM_SECONDS) \
            and is_order_brushing(concen_rate(userid_list, start_time_idx, end_time_idx), BRUSH_CONCEN_RATE): 
                
                # Update transact_dict of the shop by iterating over all the entries in the period.
                for userid, orderid in zip(userid_list[start_time_idx : end_time_idx + 1], 
                                           orderid_list[start_time_idx : end_time_idx + 1]):
                    transact_dict[userid].add(orderid)
    
    
    # Identify the suspicious buyers and update shop_dict.
    assign_suspicious(shop_dict, get_suspicious(transact_dict), shopid_list, curr_shopgrp_idx)
    
    # Clear transact_dict in preparation for the next shop.
    transact_dict.clear()

As a sanity test, we display a table containing the shop IDs, the list of suspicious buyers, and the indices of the first and last entries.

In [ ]:
results = pd.DataFrame(shop_dict.items(), columns = ['Shop ID', 'Suspicious Buyers'])
results['Index of First Entry'] = shopgrp_idx_list[:-1]
results['Index of Last Entry'] = shopgrp_idx_list[1:] - 1

results

<br>

# Create the submission file

Creating the CSV file containing the results of our data analysis is the final step in the order brushing detection challenge.

In [ ]:
with open('order_brush_submission.csv', mode = 'w', newline = '') as file:
    
    # The names of the headers are included in the specifications.
    header_names = ['shopid', 'userid']
    
    # Since the contents of the CSV file are derived from shop_dict, we use the DictWriter class,
    #     which is tailored for Python dictionaries.
    writer = csv.DictWriter(file, fieldnames = header_names)
    
    # Write the header: shopid,userid.
    writer.writeheader()
    
    # Iterate over every item in shop_dict.
    for shop, suspicious_list in shop_dict.items():
        
        # The user IDs are separated by an ampersand. There are no spaces before and after the ampersand. 
        # Since the user IDs are stored as integers, they should be converted to strings first.
        user_str = '&'.join([str(user) for user in suspicious_list])
        
        # Each row should indicate the shop ID followed by the user IDs of the suspicious buyers.
        # These two fields are separated by a comma (which is already the default). 
        writer.writerow({'shopid': shop, 'userid': user_str})